In [15]:
import requests


item_url = "https://penguin-stats.io/PenguinStats/api/v2/items"
matrix_url = "https://penguin-stats.io/PenguinStats/api/v2/result/matrix?show_closed_zones=true"
stage_url = "https://penguin-stats.io/PenguinStats/api/v2/stages"
zone_url = "https://penguin-stats.io/PenguinStats/api/v2/zones"

item_data = requests.get(item_url).json()
matrix_data = requests.get(matrix_url).json()
stage_data = requests.get(stage_url).json()
zone_data = requests.get(zone_url).json()


In [16]:
item_dict = {item_info["itemId"]: item_info for item_info in item_data}
stage_dict = {stage_info["stageId"]: stage_info for stage_info in stage_data}
zone_dict = {zone_info["zoneId"]: zone_info for zone_info in zone_data}


In [17]:
from datetime import datetime, timezone, timedelta


def get_item_type(item_id):
    return item_dict[item_id]["itemType"]


def get_item_name(item_id):
    return item_dict[item_id]["name"]


def get_item_rarity(item_id):
    return item_dict[item_id]["rarity"]


def get_item_id_by_name(item_name):
    for item_id, item_info in item_dict.items():
        if item_info["name"] == item_name:
            return item_id
    else:
        raise ValueError(f"Item name {item_name} not found.")


def get_stage_name(stage_id):
    return stage_dict[stage_id]["code"]


def get_stage_open_timestamp(stage_id, server):
    return stage_dict[stage_id]["existence"][server]["openTime"]


TIMEZONE_OFFSET_HOURS_DICT = {
    "CN": +8,
    "US": -5,
    "JP": +9,
    "KR": +9,
}


def get_timezone(server):
    return timezone(timedelta(hours=TIMEZONE_OFFSET_HOURS_DICT[server]))


def get_stage_type(stage_id):
    return stage_dict[stage_id]["stageType"]


def get_stage_zone_id(stage_id):
    return stage_dict[stage_id]["zoneId"]


def get_stage_open_time(stage_id, server):
    return datetime.fromtimestamp(get_stage_open_timestamp(stage_id, server) / 1000).astimezone(get_timezone(server))


def get_stage_ap_cost(stage_id):
    return stage_dict[stage_id]["apCost"]


def get_zone_name(zone_id):
    return zone_dict[zone_id]["zoneName"]


In [18]:
SERVER = "CN"
TIMES_THRESHOLD = 512

stage_drop_info = {stage_id: {} for stage_id in stage_dict}
for element in matrix_data["matrix"]:
    stage_id = element["stageId"]
    item_id = element["itemId"]
    times = element["times"]
    quantity = element["quantity"]
    assert item_id not in stage_drop_info[stage_id]
    stage_drop_info[stage_id][item_id] = (quantity, times)


In [19]:
import pandas as pd

records = []
for stage_id, drop_info in stage_drop_info.items():
    if get_stage_type(stage_id) != "ACTIVITY":
        continue
    drop_info_filtered = {
        item_id: (quantity, times)
        for item_id, (quantity, times) in drop_info.items()
        if times >= TIMES_THRESHOLD and get_item_type(item_id) == "MATERIAL"}
    if len(drop_info_filtered) != 1:
        continue
    stage_name = get_stage_name(stage_id)
    zone_name = get_zone_name(get_stage_zone_id(stage_id))
    stage_open_time = get_stage_open_time(stage_id, SERVER)
    ap_cost = get_stage_ap_cost(stage_id)
    (item_id, (quantity, times)), = drop_info_filtered.items()
    item_name = get_item_name(item_id)
    if get_item_rarity(item_id) != 2:
        continue
    records.append({
        '作战名称': stage_name,
        '活动名称': zone_name,
        '作战开放时间': stage_open_time,
        '作战理智消耗': ap_cost,
        '作战掉落物品名称': item_name,
        '掉落数': quantity,
        '样本数': times,
    })

df = pd.DataFrame.from_records(records)
df["作战掉落物品数量"] = df["掉落数"] / df["样本数"]
df["单位理智掉落物品数量"] = df["作战掉落物品数量"] / df["作战理智消耗"]
df["单件期望理智"] = 1 / df["单位理智掉落物品数量"]
df.to_csv("掉单一蓝材料的活动.csv")


In [20]:
from scipy import stats


def chi2_test(作战理智消耗, 掉落数, 样本数):
    p = 掉落数.sum() / (样本数 * 作战理智消耗).sum()
    return stats.chisquare(掉落数, p * 样本数 * 作战理智消耗)


In [24]:
groups = []
records = []
for item_name, group in sorted(df.groupby("作战掉落物品名称"), key=lambda x: get_item_id_by_name(x[0])):
    总掉落数 = group["掉落数"].sum()
    总样本数 = group["样本数"].sum()
    总消耗理智 = (group["样本数"] * group["作战理智消耗"]).sum()
    单位理智掉落物品数量 = 总掉落数 / 总消耗理智
    单件期望理智 = 1 / 单位理智掉落物品数量
    group["理论掉落数"] = 总掉落数 / 总消耗理智 * group["样本数"] * group["作战理智消耗"]
    chi2_result = chi2_test(group["作战理智消耗"], group["掉落数"], group["样本数"])
    records.append({
        "作战掉落物品名称": item_name,
        "总掉落数": 总掉落数,
        "总样本数": 总样本数,
        "总消耗理智": 总消耗理智,
        "单位理智掉落物品数量": 单位理智掉落物品数量,
        "单件期望理智": 单件期望理智,
        "卡方检验p值": chi2_result[1],
    })
    groups.append(group)

df_grouped = pd.concat(groups)
df_grouped.to_csv("掉单一蓝材料的活动_按物品分组.csv")

dfp = pd.DataFrame.from_records(records)
dfp


,作战掉落物品名称,总掉落数,总样本数,总消耗理智,单位理智掉落物品数量,单件期望理智,卡方检验p值
0,固源岩组,25487856,23352780,489286641,0.052092,19.196854,9.984781e-01
1,糖组,12432069,13146895,268662879,0.046274,21.610472,9.980974e-01
2,聚酸酯组,8662306,8937454,187191279,0.046275,21.609867,9.934143e-01
3,异铁组,23825115,30655795,643472121,0.037026,27.008143,1.384855e-23
4,酮凝集组,17447695,22441881,470827320,0.037058,26.985073,1.898807e-01
5,全新装置,14280116,25038611,516383202,0.027654,36.160995,1.399346e-31
6,扭转醇,22467873,24532178,512999331,0.043797,22.832572,4.377015e-32
7,轻锰矿,20437297,26371086,551416239,0.037063,26.980879,1.940804e-02
8,研磨石,17956431,27732391,557979951,0.032181,31.074101,7.698221e-15
9,RMA70-12,15204095,26122730,548067099,0.027741,36.047335,7.610640e-06
